In [26]:
import torch
p = torch.tensor([3, 4, 5.])
avg_p = torch.tensor([1, 2., 3])
#avg_p.mul_(0.999).add_(0.001, p.data)

In [23]:
avg_p = avg_p * 0.999
avg_p = avg_p + 0.001 * p.data

In [27]:
avg_p = 0.999 * avg_p + 0.001 * p.data

In [28]:
avg_p

tensor([1.0020, 2.0020, 3.0020])

In [2]:
import torch
batch_size = 4
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.FloatTensor(batch_size).fill_(1).to(device)

tensor([1., 1., 1., 1.], device='cuda:0')

In [1]:
from torch import nn
import torch

class Simple(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        
    def forward(self, x):
        return self.linear(x)

In [3]:
model = Simple()
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [4]:
model.zero_grad()

In [ ]:
def generator_loss(
    discriminator: Any,
    inception_encoder: Any,
    fake_imgs: torch.Tensor,
    real_labels: torch.Tensor,
    words_emb: torch.Tensor,
    sent_emb: torch.Tensor,
    match_labels: torch.Tensor,
    cap_lens: torch.Tensor,
    class_ids: torch.Tensor,
    vgg_encoder: Any,
    real_imgs: torch.Tensor,
    device: Any,
    const_dict: dict[Any, Any],
) -> Any:

In [1]:
import torch
mu = torch.tensor([1, 2, 3])

# KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
# KLD = torch.mean(KLD_element).mul_(-0.5)

In [9]:
mu = torch.randn(2, 10)
logvar = torch.randn(2, 10)
KLD_element = 1 + logvar - (mu ** 2 + torch.exp(logvar))
KLD = -0.5 * torch.mean(KLD_element)

In [12]:
torch.mean(KLD_element)

tensor(-1.2742)

In [4]:
mu

tensor([1, 2, 3])

In [5]:
"""Conditioning Augmentation Module"""

from typing import Any

import torch
from torch import nn


class CondAugmentation(nn.Module):
    """Conditioning Augmentation Module"""

    def __init__(self, D: int, conditioning_dim: int):
        """
        :param D: Dimension of the text embedding space [D from AttnGAN paper]
        :param conditioning_dim: Dimension of the conditioning space
        """
        super().__init__()
        self.cond_dim = conditioning_dim
        self.cond_augment = nn.Linear(D, conditioning_dim * 4, bias=True)
        self.glu = nn.GLU(dim=1)

    def encode(self, text_embedding: torch.Tensor) -> Any:
        """
        This function encodes the text embedding into the conditioning space
        :param text_embedding: Text embedding
        :return: Conditioning embedding
        """
        x_tensor = self.glu(self.cond_augment(text_embedding))
        mu_tensor = x_tensor[:, : self.cond_dim]
        logvar = x_tensor[:, self.cond_dim :]
        return mu_tensor, logvar

    def sample(self, mu_tensor: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        """
        This function samples from the Gaussian distribution
        :param mu: Mean of the Gaussian distribution
        :param logvar: Log variance of the Gaussian distribution
        :return: Sample from the Gaussian distribution
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(
            std
        )  # check if this should add requires_grad = True to this tensor?
        return mu_tensor + eps * std

    def forward(self, text_embedding: torch.Tensor) -> Any:
        """
        This function encodes the text embedding into the conditioning space,
        and samples from the Gaussian distribution.
        :param text_embedding: Text embedding
        :return c_hat: Conditioning embedding (C^ from StackGAN++ paper)
        :return mu: Mean of the Gaussian distribution
        :return logvar: Log variance of the Gaussian distribution
        """
        mu_tensor, logvar = self.encode(text_embedding)
        c_hat = self.sample(mu_tensor, logvar)
        return c_hat, mu_tensor, logvar


In [6]:
D = 10
cond_dim = 5
ca = CondAugmentation(D, cond_dim)

text_embedding = torch.randn(1, D)
c_hat, mu, logvar = ca(text_embedding)

In [7]:
mu.shape

torch.Size([1, 5])

In [8]:
logvar.shape

torch.Size([1, 5])